In [25]:
from datetime import datetime
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing

In [26]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir="tf_logs"
logdir="{}/run-{}".format(root_logdir, now)
logdir

'tf_logs/run-20181205025838'

### Supported code from the intro

In [27]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

n_epochs=1000
learning_rate=0.01

x=tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y=tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="Y")
theta=tf.Variable(tf.random_uniform([n+1, 1], -1, 1), name="theta")
y_pred=tf.matmul(x,theta, name="predictions")
error=y_pred-y

mse=tf.reduce_mean(tf.square(error), name="mse")

def fetch_batch(epoch, index, size):
    data = housing_data_plus_bias.data
    labels = housing.target
    data_batch=[housing_data_plus_bias[i] for i in range(index-1, (index + size) -1)]
    label_batch=[[housing.target[i]] for i in range(index-1, (index + size) -1)]
    return data_batch, label_batch

batch_size = 100
n_batches= int(np.ceil(m/batch_size))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_ops = optimizer.minimize(mse)

x1 = tf.placeholder(tf.float32, shape=(None, n+1), name="x")
y1 = tf.placeholder(tf.float32, shape=(None, 1), name = "y")

init=tf.global_variables_initializer()

In [28]:
mse_summary = tf.summary.scalar("MSE", mse)
filewriter = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [29]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for index in range(n_batches):
            x_batch, y_batch = fetch_batch(epoch, index, batch_size)
            if index % 10 ==0:
                summary_str = mse_summary.eval(feed_dict={x1: x_batch, y1: y_batch})
                step=epoch*n_batches + index
                filewriter.add_summary(summary_str, step)
            sess.run(training_ops, feed_dict={x1: x_batch, y1: y_batch})
    best_theta = theta.eval()

In [30]:
filewriter.close()